In [ ]:
# Bybit risk limit url 
# "https://api-testnet.bybit.com/derivatives/v3/public/risk-limit/list"
# Okx risk limit url 
# /api/v5/public/position-tiers
# "https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=BTC-USDT,ETH-USDT,XRP-USDT"
# instType=SWAP&tdMode=cross&uly=BTC-USDT,ETH-USDT,XRP-USDT 
# https://www.okx.com/docs-v5/zh/#public-data-rest-api-get-position-tiers

#获取交易产品基础信息
# GET /api/v5/public/instruments
# https://www.okx.com/docs-v5/zh/#public-data-rest-api-get-instruments

# BYBIT

In [89]:
import requests
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

url = "https://api-testnet.bybit.com/derivatives/v3/public/risk-limit/list"

response = requests.request("GET", url)

bybit = pd.DataFrame(response.json()['result']['list'])


bybit_symbols = list(bybit['symbol'].unique())
# group by symbol 
bybit.groupby(by='symbol').get_group(bybit_symbols[5])

bybit[['limit','maintainMargin','initialMargin','isLowestRisk','maxLeverage']] = bybit[['limit','maintainMargin','initialMargin','isLowestRisk','maxLeverage']].astype(float)
bybit.sort_values(by='maxLeverage',ascending=False)
# bybit_50 = bybit[bybit['maxLeverage']==50.0]

bybit[bybit['symbol'].str.endswith('USDT')]

bybit_maxLeverage = pd.DataFrame()

for symbol in bybit_symbols:
    single_max = bybit.groupby(by='symbol').get_group(symbol).sort_values(by='maxLeverage',ascending=False)
    bybit_maxLeverage = pd.concat([bybit_maxLeverage,single_max],ignore_index=True)

def check_mm(df):
    if df['initialMargin'] / df['maintainMargin']>=2:
        return True
    else:
        return round(df['initialMargin'] / df['maintainMargin'],2)
    
bybit_maxLeverage['mm_check'] = bybit_maxLeverage.apply(check_mm,axis=1)
bybit_maxLeverage = bybit_maxLeverage[bybit_maxLeverage.symbol.str.endswith('USDT')]
bybit_maxLeverage[bybit_maxLeverage['mm_check'] !=True]
bybit_funding_url = "https://api2.bybit.com/contract/v5/public/support/trading-param?category=LinearPerpetual"
funding_data = requests.request("GET",bybit_funding_url).json()['result']['list']
funding_df = pd.DataFrame(funding_data)

funding_df = funding_df[['symbolName','fundingRateInterval','fundingRateClamp']]
bybit_info = pd.merge(left= bybit_maxLeverage,right=funding_df,left_on='symbol',right_on='symbolName',how='left').drop(columns=['symbolName','id'])
bybit_info['exchange'] = 'bybit'
# bybit_info includes bracket levels in Bybit including imr, mmr, leverage,funding rate interval, funding rate clamp, max position on leverage 

# OKX 

In [37]:
import pandas as pd 
import requests 
import time 

# 获取OkX 所有合约交易对
okx_url = "https://www.okx.com/api/v5/public/instruments?instType=SWAP"
resp = requests.get(okx_url).json()['data']
okx_symbols = []
for symbol in resp:
    if 'USDT' in symbol['instId'] and symbol['state']=='live':
        okx_symbols.append(symbol['instId'])

ok_bracket = pd.DataFrame()
for i in range(0,len(okx_symbols),3):
    if i%3 ==0:
        symbol_para = (okx_symbols[i].split('-SWAP')[0]+","+okx_symbols[i+1].split('-SWAP')[0]+","+okx_symbols[i+2].split('-SWAP')[0])
        okx_riskUrl = "https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly="+symbol_para
        resp = requests.get(okx_riskUrl).json()['data']
        temp = pd.DataFrame(resp)
        ok_bracket = pd.concat([ok_bracket,temp],ignore_index=True)
        time.sleep(1)
print("Download Completed!!! ")

ok_bracket['instFamily'] = ok_bracket['instFamily'].str.replace('-',"")
group_1 = ['BTCUSDT']
group_2 = ["ADA", "AVAX", "BCH", "DOT", "EOS", "ETC", "ETH", "FIL", "LINK", "LTC", "TRX", "XRP"]
group_2 = [i+'USDT' for i in group_2]
group_2.append('BTCUSDC')

for index,column in ok_bracket.iterrows():
    if column['instFamily'] in group_1:
        ok_bracket.loc[index,'fundingCap/Floor'] = 0.375
    elif column['instFamily'] in group_2:
        ok_bracket.loc[index,'fundingCap/Floor'] = 0.75
    else:
        ok_bracket.loc[index,'fundingCap/Floor'] = 1.5

ok_bracket.drop(columns = 'baseMaxLoan')
ok_bracket['funding_interval'] = 8
ok_bracket['exchange'] ='okx'
filter_okx = ok_bracket[['instFamily','imr','mmr','maxSz','maxLever','fundingCap/Floor','funding_interval','exchange']]

# 获取okx合约标记价格
# GET /api/v5/public/mark-price?instType=SWAP

ok_mp = "https://www.okx.com/api/v5/public/mark-price?instType=SWAP"
ok_mp = pd.DataFrame(requests.get(ok_mp).json()['data'])
ok_mp['markPx'] = ok_mp['markPx'].astype(float)
ok_mp['symbol'] = ok_mp['instId'].str.replace('-SWAP','').str.replace('-','')
ok_mp['ts'] =  pd.to_datetime(ok_mp['ts'],unit='ms')
ok_mp = ok_mp[['symbol','markPx','ts']]

ts = int(time.time()*1000)
url_1 = f"https://www.okx.com/priapi/v5/public/products?t={ts}&instType=SWAP&instId=BTC-USDT-SWAP,ETH-USDT-SWAP,LTC-USDT-SWAP,XRP-USDT-SWAP,BCH-USDT-SWAP,SOL-USDT-SWAP,TRB-USDT-SWAP,STARL-USDT-SWAP,PEPE-USDT-SWAP,FIL-USDT-SWAP,AIDOGE-USDT-SWAP,1INCH-USDT-SWAP,AAVE-USDT-SWAP,ADA-USDT-SWAP,AGLD-USDT-SWAP,ALGO-USDT-SWAP,ALPHA-USDT-SWAP,ANT-USDT-SWAP,APE-USDT-SWAP,API3-USDT-SWAP,APT-USDT-SWAP,AR-USDT-SWAP,ARB-USDT-SWAP,ATOM-USDT-SWAP,AVAX-USDT-SWAP,AXS-USDT-SWAP,BADGER-USDT-SWAP,BAL-USDT-SWAP,BAND-USDT-SWAP,BAT-USDT-SWAP,BICO-USDT-SWAP,BIGTIME-USDT-SWAP,BLUR-USDT-SWAP,BNB-USDT-SWAP,BNT-USDT-SWAP,BSV-USDT-SWAP,CELO-USDT-SWAP,CEL-USDT-SWAP,CETUS-USDT-SWAP,CFX-USDT-SWAP,CHZ-USDT-SWAP,COMP-USDT-SWAP,CORE-USDT-SWAP,CRO-USDT-SWAP,CRV-USDT-SWAP,CSPR-USDT-SWAP,CVC-USDT-SWAP,DASH-USDT-SWAP,DOGE-USDT-SWAP,DOT-USDT-SWAP"
url_2 = f"https://www.okx.com/priapi/v5/public/products?t={ts}&instType=SWAP&instId=DYDX-USDT-SWAP,EGLD-USDT-SWAP,ENS-USDT-SWAP,EOS-USDT-SWAP,ETC-USDT-SWAP,ETHW-USDT-SWAP,FITFI-USDT-SWAP,FLM-USDT-SWAP,FLOKI-USDT-SWAP,FRONT-USDT-SWAP,FTM-USDT-SWAP,GALA-USDT-SWAP,GAS-USDT-SWAP,GFT-USDT-SWAP,GMT-USDT-SWAP,GMX-USDT-SWAP,GODS-USDT-SWAP,GRT-USDT-SWAP,HBAR-USDT-SWAP,ICP-USDT-SWAP,IMX-USDT-SWAP,IOST-USDT-SWAP,IOTA-USDT-SWAP,JST-USDT-SWAP,KISHU-USDT-SWAP,KLAY-USDT-SWAP,KNC-USDT-SWAP,KSM-USDT-SWAP,LDO-USDT-SWAP,LINK-USDT-SWAP,LOOKS-USDT-SWAP,LPT-USDT-SWAP,LRC-USDT-SWAP,LUNA-USDT-SWAP,LUNC-USDT-SWAP,MAGIC-USDT-SWAP,MANA-USDT-SWAP,MASK-USDT-SWAP,MATIC-USDT-SWAP,MINA-USDT-SWAP,MKR-USDT-SWAP,NEAR-USDT-SWAP,NEO-USDT-SWAP,NFT-USDT-SWAP,OMG-USDT-SWAP,ONT-USDT-SWAP,OP-USDT-SWAP,ORBS-USDT-SWAP,ORDI-USDT-SWAP,PEOPLE-USDT-SWAP"
url_3 = f"https://www.okx.com/priapi/v5/public/products?t={ts}&instType=SWAP&instId=PERP-USDT-SWAP,QTUM-USDT-SWAP,RACA-USDT-SWAP,RDNT-USDT-SWAP,REN-USDT-SWAP,RNDR-USDT-SWAP,RSR-USDT-SWAP,RVN-USDT-SWAP,SAND-USDT-SWAP,SHIB-USDT-SWAP,SLP-USDT-SWAP,SNX-USDT-SWAP,STORJ-USDT-SWAP,STX-USDT-SWAP,SUI-USDT-SWAP,SUSHI-USDT-SWAP,SWEAT-USDT-SWAP,THETA-USDT-SWAP,TON-USDT-SWAP,TRX-USDT-SWAP,UMA-USDT-SWAP,UNI-USDT-SWAP,USDC-USDT-SWAP,USTC-USDT-SWAP,VRA-USDT-SWAP,WAVES-USDT-SWAP,WAXP-USDT-SWAP,WLD-USDT-SWAP,WOO-USDT-SWAP,WSM-USDT-SWAP,XCH-USDT-SWAP,XLM-USDT-SWAP,XMR-USDT-SWAP,XTZ-USDT-SWAP,YFI-USDT-SWAP,YFII-USDT-SWAP,YGG-USDT-SWAP,ZEC-USDT-SWAP,ZEN-USDT-SWAP,ZIL-USDT-SWAP,ZRX-USDT-SWAP,BTC-USDC-SWAP,ETH-USDC-SWAP,BTC-USD-SWAP,ETH-USD-SWAP,LTC-USD-SWAP,XRP-USD-SWAP,BCH-USD-SWAP,SOL-USD-SWAP,FIL-USD-SWAP"

url_lists = [url_1,url_2,url_3]

symbol_faceValue = pd.DataFrame()

for url in url_lists:
    ok_symbol_list = requests.get(url).json()['data']
    ok_symbol_FV = pd.DataFrame([{'symbol':i.get('instFamily').replace('-',''),'face_value':i.get('ctVal')} for i in ok_symbol_list])
    ok_symbol_FV['face_value'] = ok_symbol_FV['face_value'].astype(float)
    symbol_faceValue = pd.concat([symbol_faceValue,ok_symbol_FV],ignore_index=True)
    time.sleep(0.5)
symbol_faceValue
first_merge = pd.merge(ok_mp,symbol_faceValue,on='symbol',how='right')
# 
first_merge.head()
first_merge['value'] = first_merge['markPx']*first_merge['face_value']
# pd.merge()
second_merge = pd.merge(first_merge,filter_okx,left_on='symbol',right_on='instFamily',how='right')
# second_merge
# filter_okx['maxSz'] = second_merge['value']*second_merge['maxSz']
# filter_okx
second_merge['maxSz'] = second_merge['maxSz'].astype(float)
second_merge['value']*second_merge['maxSz']
filter_okx['maxSz'] = second_merge['value']*second_merge['maxSz']
# filter_ok contains instFamily (symbol), imr, mmr, maxSz(max position on leverage in USDT), maxLever(leverage), funding cap/floor, funding interval, exchange
        

https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=BTC-USDT,ETH-USDT,LTC-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=XRP-USDT,BCH-USDT,SOL-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=TRB-USDT,STARL-USDT,FIL-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=PEPE-USDT,AIDOGE-USDT,1INCH-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=AAVE-USDT,ADA-USDT,AGLD-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=ALGO-USDT,ALPHA-USDT,ANT-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=APE-USDT,API3-USDT,APT-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=AR-USDT,ARB-USDT,ATOM-USDT
https://www.okx.com/api/v5/public/position-tiers?instType=SWAP&tdMode=cross&uly=AVAX-USDT,AXS-USDT,BADGER-USDT
https://www.okx.com

/var/folders/lc/1c06746d0jv39v72f59jc43w0000gn/T/ipykernel_1510/1665348160.py:51: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  ok_mp['ts'] =  pd.to_datetime(ok_mp['ts'],unit='ms')
/var/folders/lc/1c06746d0jv39v72f59jc43w0000gn/T/ipykernel_1510/1665348160.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_okx['maxSz'] = second_merge['value']*second_merge['maxSz']


,instFamily,imr,mmr,maxSz,maxLever,fundingCap/Floor,funding_interval,exchange
0,BTCUSDT,0.008,0.004,342340.0,125,0.375,8,okx
1,BTCUSDT,0.01,0.005,1711700.0,100,0.375,8,okx
2,BTCUSDT,0.015,0.0075,6846800.0,66.66,0.375,8,okx
3,BTCUSDT,0.02,0.0125,13693600.0,50,0.375,8,okx
4,BTCUSDT,0.025,0.0175,20540400.0,40,0.375,8,okx
...,...,...,...,...,...,...,...,...
13151,ZRXUSDT,0.48,0.445,3187332.0,2.08,1.500,8,okx
13152,ZRXUSDT,0.485,0.45,3223968.0,2.06,1.500,8,okx
13153,ZRXUSDT,0.49,0.455,3260604.0,2.04,1.500,8,okx
13154,ZRXUSDT,0.495,0.46,3297240.0,2.02,1.500,8,okx


### ok 获取合约面值数据  
https://www.okx.com/priapi/v5/public/products?t=1698692954633&instType=SWAP&instId=BTC-USDT-SWAP,ETH-USDT-SWAP,LTC-USDT-SWAP,XRP-USDT-SWAP,BCH-USDT-SWAP,SOL-USDT-SWAP,TRB-USDT-SWAP,STARL-USDT-SWAP,PEPE-USDT-SWAP,FIL-USDT-SWAP,AIDOGE-USDT-SWAP,1INCH-USDT-SWAP,AAVE-USDT-SWAP,ADA-USDT-SWAP,AGLD-USDT-SWAP,ALGO-USDT-SWAP,ALPHA-USDT-SWAP,ANT-USDT-SWAP,APE-USDT-SWAP,API3-USDT-SWAP,APT-USDT-SWAP,AR-USDT-SWAP,ARB-USDT-SWAP,ATOM-USDT-SWAP,AVAX-USDT-SWAP,AXS-USDT-SWAP,BADGER-USDT-SWAP,BAL-USDT-SWAP,BAND-USDT-SWAP,BAT-USDT-SWAP,BICO-USDT-SWAP,BIGTIME-USDT-SWAP,BLUR-USDT-SWAP,BNB-USDT-SWAP,BNT-USDT-SWAP,BSV-USDT-SWAP,CELO-USDT-SWAP,CEL-USDT-SWAP,CETUS-USDT-SWAP,CFX-USDT-SWAP,CHZ-USDT-SWAP,COMP-USDT-SWAP,CORE-USDT-SWAP,CRO-USDT-SWAP,CRV-USDT-SWAP,CSPR-USDT-SWAP,CVC-USDT-SWAP,DASH-USDT-SWAP,DOGE-USDT-SWAP,DOT-USDT-SWAP

### ok 获取标记价格

GET /api/v5/public/mark-price



# Binance

In [90]:
from binance import Client
api_key = 'WcdXAQ17VxvMohivqJaXBllZ9MXpUBBYKpwQ3vExKRk2kNcQuX0xW8FZJfWD3TUB'
api_secret = 'xmGhf6qD9oPjeuCmVcyDxoDZ5hzrsjhp5qXa1xnmYtijr2muZXLItkBrDWqXqGG6'
client = Client(api_key,api_secret)
# pd.DataFrame(client.futures_leverage_bracket())
bracket = client.futures_leverage_bracket()
bn_bracket = pd.DataFrame()
for i in bracket:
    temp = pd.DataFrame(i['brackets'])
    temp.insert(0,'symbol',i['symbol'])
    bn_bracket = pd.concat([bn_bracket,temp],ignore_index=True)

bn_funding = pd.read_excel('funding_interval.xlsx')
bn_funding[['symbol','funding_interval','current_cap_floor','default_cap_floor']]

bn_funding['adj_cap_floor'] = bn_funding['current_cap_floor'].str.split(' / ',expand=True)[0].str.replace('%','').astype(float)
bn_funding['default_cap_floor'] = bn_funding['default_cap_floor'].str.split(' / ',expand=True)[0].str.replace('%','').astype(float)
bn_funding = bn_funding[['symbol','funding_interval','adj_cap_floor','default_cap_floor']]
bn_funding['symbol'] = bn_funding['symbol'].str.replace(' Perpetual','')
bn_bracket = pd.merge(left=bn_bracket,right=bn_funding,on='symbol',how='left').dropna(subset=['funding_interval'])
bn_bracket['funding_interval'] = bn_bracket['funding_interval'].str.replace('h','').astype(int)
bn_bracket['exchange'] ='binance'
bn_bracket.insert(6,'initialMargin',bn_bracket['initialLeverage'].astype(float).apply(lambda x: 1/x))
# bn_bracket contains  bracket, initialLeverage, notionalCap and notionalFloor, maintMarginRatio, iniitalMargin, cum, funding_interval, adj_cap_floor, default_cap_floor, exchange

# 对比Bybit, OKX, Binance的合约bracket和funding 参数

### 在Binance, OKX, Bybit 对比单一symbol 

In [94]:
# 进行合并
# okx: filter_okx
# bybit: bybit_info
# binance: bn_bracket
# 统一列字段名
# symbol,imr,mmr,amount_cap,amt_floor,leverage,funding_interval,exchange
search_symbol = input("Please input the symbol you want to search: ")
columns_name = ['symbol','imr','mmr','amount_cap','leverage','funding_cap','funding_interval','exchange']
bn = bn_bracket[['symbol','initialMargin','maintMarginRatio','notionalCap','initialLeverage','adj_cap_floor','funding_interval','exchange']]
ok = filter_okx[['instFamily','imr','mmr','maxSz','maxLever','fundingCap/Floor','funding_interval','exchange']]
bybit = bybit_info[['symbol','initialMargin','maintainMargin','limit','maxLeverage','fundingRateClamp','fundingRateInterval','exchange']]

bn.columns = columns_name
ok.columns = columns_name
ok[['imr','mmr','amount_cap','leverage','funding_cap','funding_interval']] = ok[['imr','mmr','amount_cap','leverage','funding_cap','funding_interval']].astype(float)
bybit.columns = columns_name
bybit[['funding_cap','funding_interval']] = bybit[['funding_cap','funding_interval']].astype(float)

bn_symbols = list(set(bn['symbol']))
bn_highestLeverage = pd.DataFrame()
for symbol in range(len(bn_symbols)):
    temp = bn.groupby(by='symbol').get_group(bn_symbols[symbol]).sort_values(by='leverage',ascending=False).head(1)
    bn_highestLeverage = pd.concat([bn_highestLeverage,temp],ignore_index=True)
bn_highestLeverage

ok_symbols = list(set(ok['symbol']))
ok_highestLeverage = pd.DataFrame()
for symbol in range(len(ok_symbols)):
    temp = ok.groupby(by='symbol').get_group(ok_symbols[symbol]).sort_values(by='leverage',ascending=False).head(1)
    ok_highestLeverage = pd.concat([ok_highestLeverage,temp],ignore_index=True)
ok_highestLeverage

bybit_symbols = list(set(bybit['symbol']))
bybit_highestLeverage = pd.DataFrame()
for symbol in range(len(bybit_symbols)):
    temp = bybit.groupby(by='symbol').get_group(bybit_symbols[symbol]).sort_values(by='leverage',ascending=False).head(1)
    bybit_highestLeverage = pd.concat([bybit_highestLeverage,temp],ignore_index=True)

bn_symbols = bn_highestLeverage.symbol.to_list()
final_merge = pd.DataFrame()
for symbol in bn_symbols:
    if symbol in ok_highestLeverage.symbol.to_list():
        temp = ok_highestLeverage[ok_highestLeverage['symbol']==symbol]
        final_merge = pd.concat([final_merge,temp],ignore_index=True)
    if symbol in bybit_highestLeverage.symbol.to_list():
        temp = bybit_highestLeverage[bybit_highestLeverage['symbol']==symbol]
        final_merge = pd.concat([final_merge,temp],ignore_index=True)
final_merge = pd.concat([final_merge,bn_highestLeverage],ignore_index=True)
final_merge.groupby(by=['symbol']).get_group(search_symbol)


,symbol,imr,mmr,amount_cap,leverage,funding_cap,funding_interval,exchange
29,BTCUSDT,0.008,0.004,342340.0,125.0,0.375,8.0,okx
30,BTCUSDT,0.010,0.005,2000000.0,100.0,0.375,8.0,bybit
329,BTCUSDT,0.008,0.004,50000.0,125.0,0.300,8.0,binance


### 生成透视表，查看各symbol在不同exchanges 的参数表现

In [97]:
pivot_symbol = pd.pivot(data=final_merge,index=['symbol'],columns = ['exchange'],values=['leverage','mmr','imr','amount_cap','funding_cap','funding_interval']).fillna("NA")#.to_excel('maxLeverage_compare.xlsx')
pivot_symbol

leverage                  mmr                   imr              \
exchange       binance  bybit   okx binance bybit     okx binance bybit   okx   
symbol                                                                          
1000FLOKIUSDT     20.0  16.67    NA  0.0200  0.03      NA    0.05  0.06    NA   
1000LUNCUSDT      50.0   25.0    NA  0.0100  0.02      NA    0.02  0.04    NA   
1000PEPEUSDT      50.0   50.0    NA  0.0150  0.01      NA    0.02  0.02    NA   
1000SHIBUSDT      50.0     NA    NA  0.0065    NA      NA    0.02    NA    NA   
1000XECUSDT       20.0   12.5    NA  0.0200  0.04      NA    0.05  0.08    NA   
...                ...    ...   ...     ...   ...     ...     ...   ...   ...   
YGGUSDT           50.0   25.0  50.0  0.0150  0.02  0.0065    0.02  0.04  0.02   
ZECUSDT           50.0   25.0  50.0  0.0150  0.02  0.0065    0.02  0.04  0.02   
ZENUSDT           50.0   25.0  50.0  0.0150  0.02  0.0065    0.02  0.04  0.02   
ZILUSDT           25.0   25.0  50.0  0.0100  0.02  0.0065    0.04  0.04  0.02   
ZRXUSDT           50.0   12.5  50.0  0.0150  0.04  0.0065    0.02  0.08  0.02   

              amount_cap                    funding_cap             \
exchange         binance     bybit      okx     binance bybit  okx   
symbol                                                               
1000FLOKIUSDT     5000.0  100000.0       NA      3.0000   3.0   NA   
1000LUNCUSDT      5000.0  100000.0       NA      2.0000   1.5   NA   
1000PEPEUSDT      5000.0  200000.0       NA      2.0000   1.5   NA   
1000SHIBUSDT      5000.0        NA       NA      0.4875    NA   NA   
1000XECUSDT       5000.0   25000.0       NA      3.0000   3.0   NA   
...                  ...       ...      ...         ...   ...  ...   
YGGUSDT           5000.0  100000.0   5618.0      2.0000   1.5  1.5   
ZECUSDT          50000.0  100000.0  11344.0      3.0000   1.5  1.5   
ZENUSDT           5000.0  100000.0   3633.6      2.0000   1.5  1.5   
ZILUSDT           5000.0  200000.0   9610.0      3.0000   1.5  1.5   
ZRXUSDT           5000.0  100000.0   9159.0      3.0000   3.0  1.5   

              funding_interval             
exchange               binance bybit  okx  
symbol                                     
1000FLOKIUSDT              8.0   8.0   NA  
1000LUNCUSDT               8.0   8.0   NA  
1000PEPEUSDT               8.0   8.0   NA  
1000SHIBUSDT               8.0    NA   NA  
1000XECUSDT                8.0   8.0   NA  
...                        ...   ...  ...  
YGGUSDT                    4.0   8.0  8.0  
ZECUSDT                    8.0   8.0  8.0  
ZENUSDT                    8.0   8.0  8.0  
ZILUSDT                    8.0   8.0  8.0  
ZRXUSDT                    8.0   8.0  8.0  

[221 rows x 18 columns]

,id,symbol,limit,maintainMargin,initialMargin,isLowestRisk,maxLeverage
210,1,1000PEPEUSDT,200000.0,0.01,0.02,1.0,50.0
376,117,ADAUSDT,300000.0,0.01,0.02,0.0,50.0
690,1,APEUSDT,200000.0,0.01,0.02,1.0,50.0
750,1,APTUSDT,200000.0,0.01,0.02,1.0,50.0
780,1,ARBUSDT,200000.0,0.01,0.02,1.0,50.0
990,431,ATOMUSDT,200000.0,0.01,0.02,1.0,50.0
1075,446,AVAXUSDT,200000.0,0.01,0.02,1.0,50.0
1105,371,AXSUSDT,200000.0,0.01,0.02,1.0,50.0
1285,56,BCHUSDT,200000.0,0.01,0.02,1.0,50.0
1435,251,BNBUSDT,200000.0,0.01,0.02,1.0,50.0
